In [441]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.stats import zscore
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant



In [442]:
class MeanReversionStrategy:
    def __init__(self, sigma_i, kappa_i, mi, threshold_buy_open, threshold_sell_open, threshold_buy_close, threshold_sell_close):
        self.sigma_i = sigma_i  # Standard deviation of stock's returns
        self.kappa_i = kappa_i  # Mean reversion rate
        self.mi = mi  # Long term mean level of the stock's price
        self.threshold_buy_open = threshold_buy_open
        self.threshold_sell_open = threshold_sell_open
        self.threshold_buy_close = threshold_buy_close
        self.threshold_sell_close = threshold_sell_close

    def calculate_s_score(self, X_i_t):
        """
        Calculate the s-score as defined by the equation s_i = (X_i(t) - mi) / sigma_eq_i.
        X_i_t is the current price of the stock.
        """
        sigma_eq_i = self.sigma_i / np.sqrt(2 * self.kappa_i)
        s_score = (X_i_t - self.mi) / sigma_eq_i
        return s_score

    def generate_signals(self, X_i_t):
        """
        Generate trading signals based on the s-score.
        X_i_t is the current price of the stock.
        """
        s_score = self.calculate_s_score(X_i_t)
        if s_score < -self.threshold_buy_open:
            return "buy to open"
        elif s_score > self.threshold_sell_open:
            return "sell to open"
        elif s_score < self.threshold_buy_close:
            return "close short position"
        elif s_score > -self.threshold_sell_close:
            return "close long position"
        else:
            return "no action"


In [443]:
class TokenAnalysis:
    def __init__(self, returns, factors, M):
        """
        Parameters:
        - returns: A DataFrame of shape (T, 40) containing hourly returns for 40 tokens.
        - factors: A DataFrame of shape (T, 2) containing values for factors F1 and F2.
        - M: The number of historical points to consider for regression.
        """
        self.returns = returns
        self.factors = factors
        self.M = M
        self.coefficients = {}
    
    def regress_hourly_returns(self):
        """
        Apply linear regression to hourly returns to get regression coefficients.
        """
        
        # Iterate over each column (token) in the returns DataFrame
        for token in self.returns.columns:
            # Prepare the data for regression
            Y = self.returns[token][-self.M:]  # Use the token name to access the series
            X = self.factors[-self.M:]
            X = add_constant(X)  # Adds a constant term to the predictor
            
            # Perform the regression
            model = OLS(Y, X).fit()
            
            # Store the coefficients and residuals using token names as keys
            self.coefficients[token] = {
                'beta_0': model.params['const'],
                'beta_1': model.params['F1'],
                'beta_2': model.params['F2'],
                'residuals': model.resid
            }

    
    def regress_X_series(self, X_series):
        """
        Apply linear regression to the X series to get a and b coefficients.
        """
        # Prepare the data for regression
        Y = X_series[1:]  # X_{l+1}
        X = X_series[:-1]  # X_l
        X = add_constant(X)  # Adds a constant term to the predictor
        
        # Perform the regression
        model = OLS(Y, X).fit()
        
        a = model.params['const']
        b = model.params['x1']
        
        # Assuming the appendix provides a method to calculate kappa, m, sigma from a and b
        kappa, m, sigma_eq = self.calculate_from_a_b(a, b)
        
        return kappa, m, sigma_eq
    
    def calculate_from_a_b(self, a, b, xi_variance, delta_t):
        """
        Calculate values of kappa, m, sigma_eq using coefficients a and b.
        Parameters:
        a : float
            The intercept from the OU process linear regression.
        b : float
            The slope from the OU process linear regression.
        xi_variance : float
            The variance of the residuals from the OU process linear regression.
        delta_t : float
            The time increment between observations.
        """
        # Solve for kappa using b = exp(-kappa * delta_t)
        kappa = -np.log(b) / delta_t
        
        # Solve for m using a = m(1 - exp(-kappa * delta_t))
        m = a / (1 - np.exp(-kappa * delta_t))
        
        # Solve for sigma using the variance formula
        sigma = np.sqrt((xi_variance * 2 * kappa) / (1 - np.exp(-2 * kappa * delta_t)))
        
        # Calculate sigma_eq using the formula for the stationary variance of the OU process
        sigma_eq = sigma / np.sqrt(2 * kappa)
        
        return kappa, m, sigma, sigma_eq

# Example usage:
# Assuming we have loaded our returns and factors into DataFrames `hourly_returns` and `hourly_factors`
# And we have an M value defined for the number of historical points to use
# token_analysis = TokenAnalysis(hourly_returns, hourly_factors, M)
# token_analysis.regress_hourly_returns()
# X_series is some series for which we want to apply the second regression
# kappa, m, sigma_eq = token_analysis.regress_X_series(X_series)


In [444]:
def calculate_factor_returns(eigenportfolios, standardized_returns):
    # The function will return a 2x14015 matrix, where each row represents one of the two factors,
    # and each column represents the factor return at a specific hour.
    
    # Matrix multiplication of the eigenportfolio weights with the hourly returns
    # eigenportfolios is 2x38, standardized_returns.T is 38x14015
    # The resulting matrix multiplication will be 2x14015
    factor_returns = np.dot(eigenportfolios, standardized_returns.T)
    
    return factor_returns




In [445]:
class CoinDataProcessor:
    def __init__(self, coin_df, coin_names=None):
        """
        Constructs all the necessary attributes for the CoinDataProcessor object.
        
        Parameters:
        coin_df : pd.DataFrame
            A DataFrame with each column representing a different token and each row representing a different time point.
        coin_names : list, optional
            A list of column names to filter from the DataFrame.
        """
        self.coin_df = self.prepare_data(coin_df)

        # If a list of coin names is provided, filter the DataFrame to these columns
        if coin_names:
            valid_coin_names = [name for name in coin_names if name in self.coin_df.columns]
            self.coin_df = self.coin_df[valid_coin_names]

        # Exclude 'time' and 'startTime' columns and consider all others as token columns
        self.token_columns = [col for col in self.coin_df.columns if col not in ('time', 'startTime')]

    def prepare_data(self, df):
        """
        Prepares the coin data by handling NaN values.
        """
        df = df.dropna(axis=1, how='all')
        df.ffill(axis=0, inplace=True)
        return df

    def calculate_returns(self):
        """Calculates simple returns for the token data."""
        self.returns = self.coin_df[self.token_columns].pct_change()
        return self.returns
    
    def calculate_cumulative_returns(self):
        """Calculates cumulative returns for the token data."""
        if not hasattr(self, 'returns'):
            self.calculate_returns()
        self.cumulative_returns = (1 + self.returns).cumprod() - 1
        return self.cumulative_returns

    def calculate_avg_std(self):
        """
        Calculates the average and standard deviation of the returns for each token,
        filtering out inf, -inf, and NaN values.
        """
        if not hasattr(self, 'returns'):
            self.calculate_returns()

        # Replace inf and -inf with NaN, then drop NaN values
        filtered_returns = self.returns.replace([np.inf, -np.inf], np.nan).dropna()

        self.mean_returns = filtered_returns.mean()
        self.std_returns = filtered_returns.std()
        return self.mean_returns, self.std_returns

    # ... [other methods remain unchanged]

    def calculate_standardized_returns(self):
        """Calculates standardized returns for the token data."""
        if not hasattr(self, 'mean_returns') or not hasattr(self, 'std_returns'):
            self.calculate_avg_std()
        self.standardized_returns = (self.returns - self.mean_returns) / self.std_returns
        return self.standardized_returns


In [454]:
df = pd.read_csv('coin_all_prices_full.csv')

df['startTime'] = df['startTime'].str.replace(r'\+.*', '', regex=True).str.strip()
df.set_index('startTime')

top_40_df = pd.read_csv('coin_universe_150K_40.csv')

# Initialize an empty dictionary
top_40_dict = {}

for datetime_index in df.index:
    # Filter the tokens with non-NaN values at the current datetime index
    valid_tokens = df.loc[datetime_index].notna()

    # Calculate the percentage of non-NaN values for each token
    valid_tokens_percentages = df[valid_tokens.index].notna().mean() * 100

    # Select tokens with at least 80% non-NaN values
    valid_tokens_80 = valid_tokens_percentages[valid_tokens_percentages >= 80]

    # Get the top 40 tokens, if there are that many
    top_40_tokens = valid_tokens_80.nlargest(40).index.tolist()

    # Add the list of top 40 tokens to the dictionary with the datetime index as key
    top_40_dict[datetime_index] = top_40_tokens


M = 240

top_40_dict

In [447]:
# Use the class to process the stock data (retrieving top 40 companies)
processor = CoinDataProcessor(df, top_40_list)

# Calculate the standardized returns
standardized_returns = processor.calculate_standardized_returns()
standard_devs = processor.calculate_avg_std()
standardized_returns.dropna(axis = 1, how = 'all', inplace = True)


standardized_returns.fillna(0, axis = 0, inplace = True)
standardized_returns['startTime'] = pd.to_datetime(df['startTime'])
standardized_returns = standardized_returns.set_index('startTime')
standardized_returns.index = standardized_returns.index.tz_localize(None)
standardized_returns.head()



,FXS,SNX,HNT,PERP,SUSHI,BADGER,ALGO,TOMO,RSR,ZRX,...,ASD,C98,HT,BTC,DAWN,STEP,XAUT,BIT,PUNDIX,UNI
startTime,,,,,,,,,,,,,,,,,,,,,
2021-02-19 05:00:00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2021-02-19 06:00:00,0.0,0.220610,0.323952,0.319833,1.219075,0.730902,0.0,0.120763,1.212191,0.0,...,-4.629575,0.0,2.267610,1.171745,0.0,0.0,-0.016536,0.0,0.0,-0.538978
2021-02-19 07:00:00,0.0,1.023009,0.692345,1.197598,-0.586144,1.280868,0.0,2.529698,2.158279,0.0,...,1.806429,0.0,-1.766729,0.649580,0.0,0.0,0.096834,0.0,0.0,0.841327
2021-02-19 08:00:00,0.0,-0.109504,0.427285,0.369218,-0.953523,-1.141440,0.0,1.328350,0.418060,0.0,...,1.864825,0.0,-0.069514,0.113344,0.0,0.0,-0.961189,0.0,0.0,-0.991445
2021-02-19 09:00:00,0.0,-0.758828,0.028169,-0.007784,1.289863,-0.093980,0.0,-0.203140,0.175370,0.0,...,1.115304,0.0,0.709368,1.529461,0.0,0.0,0.021253,0.0,0.0,0.829323


In [450]:

top_components = []
eigenvalues_list = []

for i in range(len(standardized_returns) - M + 1):
    window = standardized_returns.iloc[i: i + M]

    corr_matrix = window.corr()
    pca = PCA(n_components=2)
    pca.fit(corr_matrix)

    components = pca.components_  # Eigenvectors
    top_components.append(components.flatten())  # Flatten and append

    eigenvalues = pca.explained_variance_  # Eigenvalues
    eigenvalues_list.append(eigenvalues)

# Convert the components to a DataFrame
df_top_components = pd.DataFrame(top_components, columns=['PC1_1', 'PC1_2', 'PC2_1', 'PC2_2'])

# Print the results
print("Top principal components (eigenvectors):")
print(df_top_components)
print("\nCorresponding eigenvalues:")
print(eigenvalues_list)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [449]:
eigenportfolios = top_components / standard_devs

factor_returns = calculate_factor_returns(eigenportfolios, standardized_returns)

factor_returns_df = pd.DataFrame(factor_returns.T, columns=['F1', 'F2'])

factor_returns_df['F1'].to_csv('task1a_1.csv')
factor_returns_df['F2'].to_csv('task1a_2.csv')

token_analysis = TokenAnalysis(standardized_returns, factor_returns_df, M)


token_analysis.regress_hourly_returns()

TypeError: unsupported operand type(s) for /: 'list' and 'tuple'

In [ ]:
# Initialize a dictionary to store the parameters for each token
token_params = {}

for i, token in enumerate(standardized_returns.columns):
    # Extract the residuals for the current token from the coefficients dictionary
    residuals = token_analysis.coefficients[token]['residuals']

    # Calculate X_l as the cumulative sum of residuals up to a certain lag l for the current token
    X_l = np.cumsum(residuals, axis=0)

    # Prepare the shifted version of X_l for the regression
    X_l_plus_1 = np.roll(X_l, -1)
    X_l = X_l[:-1]  # Remove the last element since it doesn't have a corresponding future value
    X_l_plus_1 = X_l_plus_1[1:]  # Remove the first element which is now the last element of X_l

    # Create the X DataFrame with a named column for the predictor
    X = pd.DataFrame({
        'const': np.ones(len(X_l)),
        token: X_l  # The predictor column is named after the token
    })
    Y = X_l_plus_1

    # Perform the regression
    model = OLS(Y, X).fit()

    # Access the coefficients using the token name
    a = model.params['const']
    b = model.params[token]  # Use the token name for the predictor coefficient

    # Calculate the variance of the residuals for the current token
    xi_variance = np.var(residuals)

    # Calculate delta_t for hourly data (assuming 8760 hours in a year)
    delta_t = 1 / 8760

    # Use the class's method to calculate the parameters
    kappa, m, sigma, sigma_eq = token_analysis.calculate_from_a_b(a, b, xi_variance, delta_t)

    # Store the parameters for the current token
    token_params[token] = {
        'kappa': kappa,
        'm': m,
        'sigma': sigma,
        'sigma_eq': sigma_eq
    }




In [ ]:
# Extract 'kappas' and 'sigmas' dictionaries
kappas = {token: params['kappa'] for token, params in token_params.items()}
sigmas = {token: params['sigma'] for token, params in token_params.items()}
ms = {token: params['m'] for token, params in token_params.items()}

In [ ]:

# Define thresholds
threshold_buy_open = 1.25
threshold_sell_open = 1.25
threshold_buy_close = 0.75
threshold_sell_close = 0.5

# Dictionary to store the trading signals for each token
trading_signals = {}

# Loop through each token
for i, token in enumerate(standardized_returns.columns):
    # Instantiate the strategy with the parameters for the current token
    strategy = MeanReversionStrategy(sigma_i=sigmas[token], kappa_i=kappas[token],
                                     mi=ms[token], threshold_buy_open=threshold_buy_open,
                                     threshold_sell_open=threshold_sell_open,
                                     threshold_buy_close=threshold_buy_close,
                                     threshold_sell_close=threshold_sell_close)
    
    # Calculate the s-score using the current price/return
    current_price_or_return = df[token].iloc[-1]  # Assuming the last row is the current time t
    s_score = strategy.calculate_s_score(current_price_or_return)
    
    # Generate the trading signal
    signal = strategy.generate_signals(current_price_or_return)
    
    # Store the signal
    trading_signals[token] = signal

# `trading_signals` now contains the trading signal for each token




In [ ]:
# Initialize portfolio
portfolio = {token: 0 for token in standardized_returns.columns}  # Number of shares for each token
portfolio_value = 0  # Total value of the portfolio
portfolio_values_over_time = []

# Convert the date strings to pandas Timestamps
tstart = pd.Timestamp('2021-09-26 00:00:00')#.tz_localize(None)
tend = pd.Timestamp('2022-09-25 23:00:00')#.tz_localize(None)

# Assuming the DataFrame with the 'startTime' column is named 'df'
# and the 'startTime' column is of type datetime (if not, you should convert it)

# Set 'startTime' as the index if it's not already the index
standardized_returns['startTime'] = pd.to_datetime(df['startTime'])
standardized_returns = standardized_returns.set_index('startTime')
standardized_returns.index = standardized_returns.index.tz_localize(None)


# Now 'df' is indexed by datetime, so we can slice using tstart and tend
df_period = standardized_returns.loc[tstart:tend]

# Get the indices for the starting and ending points
tstart_idx = df_period.index.get_loc(tstart)
tend_idx = df_period.index.get_loc(tend) + 1  # +1 to include the end time in the loop



In [ ]:

for t in range(tstart_idx, tend_idx):
    # Get the current prices of each token at time t
    current_prices = standardized_returns.iloc[t]
    
    # Execute trades according to signals generated
    for token in portfolio:
        signal = trading_signals[token]  # Signal generated for the token at time t
        
        # Buy or sell according to the signal
        if signal == 'buy to open' and portfolio[token] == 0:
            portfolio[token] += 1  # Buy 1 share
            portfolio_value -= current_prices[token]  # Deduct the cost of 1 share from portfolio value
        elif signal == 'sell to open' and portfolio[token] == 0:
            portfolio[token] -= 1  # Sell 1 share short
            portfolio_value += current_prices[token]  # Add the value of 1 share to portfolio value
        elif signal == 'close long position' and portfolio[token] > 0:
            portfolio_value += current_prices[token]  # Sell 1 share to close long position
            portfolio[token] -= 1
        elif signal == 'close short position' and portfolio[token] < 0:
            portfolio_value -= current_prices[token]  # Buy 1 share to close short position
            portfolio[token] += 1
    
    # Calculate the new portfolio value based on current prices
    new_portfolio_value = sum(current_prices[token] * shares for token, shares in portfolio.items())
    
    # Record the portfolio value for the current time period
    portfolio_values_over_time.append(new_portfolio_value + portfolio_value)

# Calculate the portfolio returns
portfolio_returns = [j-i for i, j in zip(portfolio_values_over_time[:-1], portfolio_values_over_time[1:])]


C:\Users\Cal.Kramer\AppData\Local\Temp\ipykernel_37428\3534831788.py:24: RuntimeWarning: invalid value encountered in double_scalars
  new_portfolio_value = sum(current_prices[token] * shares for token, shares in portfolio.items())


In [ ]:
portfolio_values_over_time

[0.0,
 -46.51707887220905,
 3.562578480932135,
 -2.584796419961202,
 -2.85241738238062,
 -10.719356654180311,
 -23.004678575317552,
 -21.45646699430337,
 -11.114689185009114,
 -29.398071661848114,
 -17.329343038067467,
 -13.112306613112992,
 12.69746200770994,
 -44.739768193832866,
 -10.480276897690123,
 10.13419313892152,
 3.1992014898604966,
 -13.349446683428768,
 -33.485839984899116,
 -51.58387575230158,
 38.83610305358869,
 13.52885772758913,
 -36.49834988480061,
 13.696778105207748,
 -2.973369744808563,
 -17.41961462213793,
 -30.245102620445294,
 51.16177391209784,
 -29.403315770716496,
 -34.788262640332,
 -3.5892771147952574,
 -35.93220032062973,
 -15.116173791959802,
 6.349957772526478,
 78.93595672121954,
 11.218554802749775,
 -46.03667395996212,
 -7.692890289193111,
 -2.5672536046856615,
 6.729976350015997,
 19.74566235232797,
 30.656308694884462,
 89.58535683554325,
 -20.696002977348698,
 -63.681127655957525,
 -20.748668069153826,
 1.15546434418234,
 15.052861693393307,
 -8.5

In [ ]:

standardized_return_outtest = standardized_returns[tstart:tend]


# 1. Calculate the empirical correlation matrix
correlation_matrix = standardized_return_outtest.corr()





In [ ]:

Roll_Max = SPY_Dat['Adj Close'].rolling(M, min_periods=1).max()
Daily_Drawdown = SPY_Dat['Adj Close']/Roll_Max - 1.0

# Next we calculate the minimum (negative) daily drawdown in that window.
# Again, use min_periods=1 if you want to allow the expanding window
Max_Daily_Drawdown = Daily_Drawdown.rolling(window, min_periods=1).min()

# Plot the results
Daily_Drawdown.plot()
Max_Daily_Drawdown.plot()
pp.show()

NameError: name 'SPY_Dat' is not defined